In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install git+https://git@github.com/ping/instagram_private_api.git@1.6.0

  Cloning https://****@github.com/ping/instagram_private_api.git (to revision 1.6.0) to /tmp/pip-req-build-ibh4c_vt
  Running command git clone -q 'https://****@github.com/ping/instagram_private_api.git' /tmp/pip-req-build-ibh4c_vt
  Running command git checkout -q d1180fc00398dc58c9903b97be0c9148d69d21ec
  Created wheel for instagram-private-api: filename=instagram_private_api-1.6.0-cp36-none-any.whl size=76470 sha256=a5480f03629e78b96750c3cfdcfff46be2c4b9c50bf841d4cfdeed0a424a4ee3
  Stored in directory: /tmp/pip-ephem-wheel-cache-x7sjeu7b/wheels/49/a5/9a/500d6a722f369125e169bcd77617c9943f6961338fe8ac83bb
Successfully built instagram-private-api


In [3]:
from random import choice
import json

import requests
from bs4 import BeautifulSoup

_user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'
]


class InstagramScraper:

    def __init__(self, user_agents=None, proxy=None):
        self.user_agents = user_agents
        self.proxy = proxy

    def __random_agent(self):
        if self.user_agents and isinstance(self.user_agents, list):
            return choice(self.user_agents)
        return choice(_user_agents)

    def __request_url(self, url):
        try:
            response = requests.get(url, headers={'User-Agent': self.__random_agent()}, proxies={'http': self.proxy,
                                                                                                 'https': self.proxy})
            response.raise_for_status()
        except requests.HTTPError:
            raise requests.HTTPError('Received non 200 status code from Instagram')
        except requests.RequestException:
            raise requests.RequestException
        else:
            return response.text

    @staticmethod
    def extract_json_data(html):
        soup = BeautifulSoup(html, 'html.parser')
        body = soup.find('body')
        script_tag = body.find('script')
        raw_string = script_tag.text.strip().replace('window._sharedData =', '').replace(';', '')
        return json.loads(raw_string)

    def profile_page_metrics(self, profile_url):
        results = {}
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']
        except Exception as e:
            raise e
        else:
            for key, value in metrics.items():
                if key != 'edge_owner_to_timeline_media':
                    if value and isinstance(value, dict):
                        value = value['count']
                        results[key] = value
                    elif value:
                        results[key] = value
        return results

    def profile_page_recent_posts(self, profile_url):
        results = []
        try:
            response = self.__request_url(profile_url)
            json_data = self.extract_json_data(response)
            metrics = json_data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']["edges"]
        except Exception as e:
            raise e
        else:
            for node in metrics:
                node = node.get('node')
                if node and isinstance(node, dict):
                    results.append(node)
        return results

In [4]:
from pprint import pprint 
k = InstagramScraper()
results = k.profile_page_recent_posts('https://www.instagram.com/realdonaldtrump/?hl=en')
pprint(results)

[{'__typename': 'GraphVideo',
  'comments_disabled': False,
  'dimensions': {'height': 640, 'width': 640},
  'display_url': 'https://scontent-iad3-1.cdninstagram.com/v/t51.2885-15/e35/81515327_106028780832110_4413543103126848812_n.jpg?_nc_ht=scontent-iad3-1.cdninstagram.com&_nc_cat=1&_nc_ohc=9kM8FK6UlR8AX9HwUNz&oh=c832c6eb4b942964fea529fcdf140d57&oe=5E18C8D0',
  'edge_liked_by': {'count': 652861},
  'edge_media_preview_like': {'count': 652861},
  'edge_media_to_caption': {'edges': [{'node': {'text': '#Repost @senategop\n'
                                                        '・・・\n'
                                                        'Qassem Soleimani’s '
                                                        '20-year span of '
                                                        'terror is finally '
                                                        'over. \u2063\n'
                                                        '\u2063\n'
                                      

In [5]:
len(results)

12